In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Attention, Dropout, BatchNormalization
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import kerastuner as kt
import random

# Set random seed for reproducibility
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

# Load Dataset
dataset_path = '/home/hoangbaoan1901/Documents/information-system_UET/sem5/prdan/tianen101/datasets/jrfm-2156907-cleaned.csv'
df = pd.read_csv(dataset_path)

# Define the target variable
target = 'BTC_Close'

# Convert date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Define new training and testing periods
start_day_p2 = pd.Timestamp(2018, 10, 1)
test_day_p2 = pd.Timestamp(2021, 10, 1)
end_day_p2 = pd.Timestamp(2022, 4, 1)

# Filter data for the specified period
period2 = df[(df['Date'] <= end_day_p2)].reset_index(drop=True)
train_dataset_p2 = period2[period2['Date'] < test_day_p2].reset_index(drop=True)
test_dataset_p2 = period2[period2['Date'] >= test_day_p2].reset_index(drop=True)
train_dataset_p2 = train_dataset_p2.drop(columns=['Date'])
test_dataset_p2 = test_dataset_p2.drop(columns=['Date'])

# Create the feature sets
X_train = train_dataset_p2[:-1]
X_test = test_dataset_p2[:-1]

# Create the target sets
Y_train = train_dataset_p2[target][1:]
Y_test = test_dataset_p2[target][1:]

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(df[target].values.reshape(-1, 1))

# Prepare data for GRU
sequence_length = 60
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(x), np.array(y)

x, y = create_sequences(data_normalized, sequence_length)

# Split into training and testing data based on date
test_index = len(train_dataset_p2) - sequence_length
x_train = x[:test_index]
y_train = y[:test_index]
x_test = x[test_index:]
y_test = y[test_index:]

# Hyperparameter Optimization using Keras Tuner
def build_model(hp):
    inputs = Input(shape=(sequence_length, 1))
    # Hyperparameter tuning for the number of units and dropout rates
    gru_output = GRU(hp.Int('units', min_value=32, max_value=256, step=32), return_sequences=False)(inputs)
    gru_output = BatchNormalization()(gru_output)
    gru_output = Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1))(gru_output)
    
    dense_output = Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu')(gru_output)
    dense_output = Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1))(dense_output)
    dense_output = Dense(1)(dense_output)

    # Compile GRU model
    model = Model(inputs, dense_output)
    model.compile(optimizer='adam', loss='mse')
    return model

# Hyperparameter tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='gru_xgb_tuner',
    project_name='gru_xgb'
)

# Early Stopping Callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Search for best hyperparameters
tuner.search(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[early_stopping])

# Get the best hyperparameters
gru_best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print('Best GRU Hyperparameters:')
print(gru_best_hyperparameters.values)

# Get the best model
gru_model = tuner.get_best_models(num_models=1)[0]

# Train the best GRU model
gru_model.fit(x_train, y_train, epochs=50, batch_size=16, validation_split=0.1, callbacks=[early_stopping])

# Extract features from GRU model
feature_extractor = Model(inputs=gru_model.input, outputs=gru_model.layers[-3].output)
train_features = feature_extractor.predict(x_train)
test_features = feature_extractor.predict(x_test)

# Set up hyperparameter optimization for XGBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.005, 0.01, 0.05, 0.1],
    'subsample': [0.6, 0.7, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1.0]
}

xgb_model = xgb.XGBRegressor()
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, n_iter=20, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Train XGBoost with optimized hyperparameters
random_search.fit(train_features, y_train)

# Print best hyperparameters for XGBoost
print('Best XGBoost Hyperparameters:')
print(random_search.best_params_)
xgb_best = random_search.best_estimator_

# Make predictions
y_pred = xgb_best.predict(test_features)

# Rescale predictions
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluation
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
print(f'RMSE: {rmse}')

# Plot the predictions against the actual values
plt.figure(figsize=(14, 7))
plt.plot(y_test_rescaled, color='blue', label='Actual BTC_Close')
plt.plot(y_pred_rescaled, color='red', label='Predicted BTC_Close', linestyle='dashed')
plt.title('BTC_Close Price Prediction')
plt.xlabel('Time')
plt.ylabel('BTC_Close Price')
plt.legend()
plt.show()

Trial 2 Complete [00h 00m 21s]
val_loss: 0.003270822111517191

Best val_loss So Far: 0.003270822111517191
Total elapsed time: 00h 00m 53s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
224               |96                |units
0.3               |0.3               |dropout
128               |96                |dense_units
0.2               |0.3               |dropout_2

Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0599 - val_loss: 0.3596
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0118 - val_loss: 0.3121
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0054 - val_loss: 0.3420
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0039 - val_loss: 0.3059
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0032 - val_loss: 0.2554
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0022 - val_loss: 0.2423
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0017 - val_loss: 0.223